In [32]:
import pandas as pd
total_data = pd.DataFrame()

## 자료 가져오기

In [33]:
jym_data = pd.read_csv("../data/전주시_체육시설_공시지가.csv")
local_tax = pd.read_csv("../data/전주시_지방세.csv")
house = pd.read_csv("../data/전주시_지역별_주택평균가격.csv")
exRatio = pd.read_csv("../data/전주시_지역별_자주_운동_비율.csv")
people = pd.read_csv("../data/전주시_동별세대_및_인구.csv", encoding='cp949')

In [34]:
total_data = jym_data.drop(columns=['Unnamed: 0', '구주소', '신주소', '주소', '본번', '부번'])

In [35]:
total_data = pd.merge(total_data, exRatio, on='시군구', how='right')

In [36]:
total_data = pd.merge(total_data, local_tax, on='시군구', how='right')

In [37]:
total_data = pd.merge(total_data, house, on='시군구', how='right')

In [38]:
people2 = people.drop(columns=['시군구'])

In [39]:
total_data = pd.merge(total_data, people2, on='읍면동', how='left')

In [40]:
total_data[total_data.isna().any(axis=1)]

,시설장명,시설면적,시군구,읍면동,공시지가,자주_운동하는_비율,지방세,주택평균가격,인구 (명),65세이상고령자 (명),인구밀도 (명/㎢)
4,완산생활체육공원 축구장,7140.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
5,완산생활체육공원 유소년축구장,4320.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
6,완산생활체육공원 야외테니스장,697.5269,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
7,완산생활체육공원 배구장,608.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
8,완산생활체육공원 배드민턴장,1717.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
9,완산생활체육공원 농구장,828.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
10,완산생활체육공원 인공암벽장,3017.0000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
11,완산생활체육공원 족구장,1592.5000,완산구,중인동,70700.0,40.466667,389300000000,175763.109,NaN,NaN,NaN
13,월드컵경기장,473293.0000,덕진구,반월동,331000.0,35.900000,346000000000,182321.891,NaN,NaN,NaN
14,월드컵골프장,154139.0000,덕진구,장동,331000.0,35.900000,346000000000,182321.891,NaN,NaN,NaN


In [41]:
# 지역구별 인구와 65세 이상 고령자 평균 계산 후 반올림하여 정수형으로 변환
grouped = total_data.groupby('시군구')[['인구 (명)', '65세이상고령자 (명)']].mean().round().astype('Int64')

# NaN 값을 지역구별 평균으로 대체하고 반올림하는 함수 정의
def fill_na_with_mean_and_round(row):
    if pd.isna(row['인구 (명)']):
        row['인구 (명)'] = grouped.loc[row['시군구'], '인구 (명)']
    if pd.isna(row['65세이상고령자 (명)']):
        row['65세이상고령자 (명)'] = grouped.loc[row['시군구'], '65세이상고령자 (명)']
    return row

# 각 행에 대해 NaN 값을 지역구별 평균으로 대체하고 반올림
total_data = total_data.apply(fill_na_with_mean_and_round, axis=1)

In [42]:
total_data = total_data.drop(columns=['인구밀도 (명/㎢)'])

<다중회귀모형>
|변수명 | 변수의 역할 |
|---|----------|
|y  | 시설면적 |
|x1 | 해당 시설의 공시지가 |
|x2 | 자주 운동하는 인구 비율 |
|x3 | 해당 구의 지방세 |
|x4 | 해당 구의 주택평균가격 |
|x5 | 해당 동의 인구 |
|x6 | 해당 동의 고령자 명수 |

In [43]:
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
total_data = total_data.drop(columns=['시설장명', '시군구', '읍면동']).apply(pd.to_numeric, errors='coerce')
cor_matrix = total_data.corr()

In [55]:
cor_matrix

,시설면적,공시지가,자주_운동하는_비율,지방세,주택평균가격,인구 (명),65세이상고령자 (명)
시설면적,1.000000,0.086397,-0.203016,-0.203016,0.203016,0.149191,0.187909
공시지가,0.086397,1.000000,-0.285368,-0.285368,0.285368,-0.014173,0.318212
자주_운동하는_비율,-0.203016,-0.285368,1.000000,1.000000,-1.000000,-0.729747,-0.897445
지방세,-0.203016,-0.285368,1.000000,1.000000,-1.000000,-0.729747,-0.897445
주택평균가격,0.203016,0.285368,-1.000000,-1.000000,1.000000,0.729747,0.897445
인구 (명),0.149191,-0.014173,-0.729747,-0.729747,0.729747,1.000000,0.519176
65세이상고령자 (명),0.187909,0.318212,-0.897445,-0.897445,0.897445,0.519176,1.000000


In [46]:
y = total_data['시설면적']
X = total_data.drop(columns=['시설면적'])

In [47]:
# 전진 선택법
def forward_selection(X, y):
    initial_features = []
    remaining_features = list(X.columns)
    selected_features = initial_features[:]
    best_score = np.inf
    
    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            features = selected_features + [candidate]
            X_new = X[features]
            X_new = sm.add_constant(X_new)
            model = sm.OLS(y, X_new).fit()
            score = mean_squared_error(y, model.predict(X_new))
            scores_with_candidates.append((score, candidate))
        
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_candidate)
            remaining_features.remove(best_candidate)
        else:
            break
    return selected_features

In [48]:
# 후진 제거법
def backward_elimination(X, y):
    features = list(X.columns)
    best_features = features[:]
    best_score = np.inf
    
    while features:
        scores_with_candidates = []
        for candidate in features:
            remaining_features = features[:]
            remaining_features.remove(candidate)
            X_new = X[remaining_features]
            X_new = sm.add_constant(X_new)
            model = sm.OLS(y, X_new).fit()
            score = mean_squared_error(y, model.predict(X_new))
            scores_with_candidates.append((score, candidate))
        
        scores_with_candidates.sort()
        best_new_score, worst_candidate = scores_with_candidates[0]
        
        if best_new_score < best_score:
            best_score = best_new_score
            features.remove(worst_candidate)
            best_features = features[:]
        else:
            break
    return best_features

In [49]:
# 단계별 선택법
def stepwise_selection(X, y):
    initial_features = []
    remaining_features = list(X.columns)
    selected_features = initial_features[:]
    best_score = np.inf
    
    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            features = selected_features + [candidate]
            X_new = X[features]
            X_new = sm.add_constant(X_new)
            model = sm.OLS(y, X_new).fit()
            score = mean_squared_error(y, model.predict(X_new))
            scores_with_candidates.append((score, candidate))
        
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.append(best_candidate)
            remaining_features.remove(best_candidate)
        else:
            break
        
        scores_with_candidates = []
        for candidate in selected_features:
            features = selected_features[:]
            features.remove(candidate)
            X_new = X[features]
            X_new = sm.add_constant(X_new)
            model = sm.OLS(y, X_new).fit()
            score = mean_squared_error(y, model.predict(X_new))
            scores_with_candidates.append((score, candidate))
        
        scores_with_candidates.sort()
        best_new_score, worst_candidate = scores_with_candidates[0]
        
        if best_new_score < best_score:
            best_score = best_new_score
            selected_features.remove(worst_candidate)
            remaining_features.append(worst_candidate)
    
    return selected_features

In [50]:
forward_selected_features = forward_selection(X, y)
backward_selected_features = backward_elimination(X, y)
stepwise_selected_features = stepwise_selection(X, y)

In [51]:
# 최적 회귀 모델 생성 및 결과 출력
X_forward = sm.add_constant(X[forward_selected_features])
model_forward = sm.OLS(y, X_forward).fit()
print(model_forward.summary())

                            OLS Regression Results                            
Dep. Variable:                   시설면적   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                 -0.105
Method:                 Least Squares   F-statistic:                    0.2884
Date:                Sun, 26 May 2024   Prob (F-statistic):              0.883
Time:                        20:31:21   Log-Likelihood:                -395.65
No. Observations:                  31   AIC:                             801.3
Df Residuals:                      26   BIC:                             808.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1.758e+05   1.34e+06      0.132   

In [52]:
X_backward = sm.add_constant(X[backward_selected_features])
model_backward = sm.OLS(y, X_backward).fit()
print(model_backward.summary())

                            OLS Regression Results                            
Dep. Variable:                   시설면적   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                 -0.105
Method:                 Least Squares   F-statistic:                    0.2884
Date:                Sun, 26 May 2024   Prob (F-statistic):              0.883
Time:                        20:31:21   Log-Likelihood:                -395.65
No. Observations:                  31   AIC:                             801.3
Df Residuals:                      26   BIC:                             808.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1.728e+05   1.32e+06      0.131   

In [53]:
X_stepwise = sm.add_constant(X[stepwise_selected_features])
model_stepwise = sm.OLS(y, X_stepwise).fit()
print(model_stepwise.summary())

                            OLS Regression Results                            
Dep. Variable:                   시설면적   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                 -0.105
Method:                 Least Squares   F-statistic:                    0.2884
Date:                Sun, 26 May 2024   Prob (F-statistic):              0.883
Time:                        20:31:22   Log-Likelihood:                -395.65
No. Observations:                  31   AIC:                             801.3
Df Residuals:                      26   BIC:                             808.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1.758e+05   1.34e+06      0.132   